In [1]:
import pandas as pd
import tabula
columns = ['state', 'river-basin', 'river', 'no-of-stations', '2019-IKA/WQI', '2019-category', '2019-class', '2020-IKA/WQI', '2020-category', '2020-class']
def extract_table(df:pd.DataFrame) -> pd.DataFrame:
    # forward fill only first column
    df.iloc[:,[0,1]] = df.iloc[:,[0,1]].ffill()
    # replace \r with space
    df = df.replace('\r','', regex=True)
    # find the first row of first column with value 'NEGERI/STATE'
    # row = df[df.iloc[:,0] == 'NEGERI /STATE'].index[0]
    # make row 0 as header
    df.columns = columns
    # drop row 0 & 1
    # df = df.drop(df.index[range(row)])
    # reset index
    df = df.reset_index(drop=True)
    return df

In [2]:
pages = [59] # jadual 2.3 Tercemar
dfs = []
for page in pages:
    df = tabula.read_pdf('EQR-2020-1.pdf', pages=[page], lattice=True,stream=True, pandas_options={'header': None})[0]
    dfs.append(extract_table(df))
df = pd.concat(dfs).reset_index(drop=True)

In [3]:
# drop row 0,1, 35,36
df = df.drop(df.index[[0,1]]).reset_index(drop=True)
# # remove rows with column 1 value equal to 'LEMBANGANSUNGAVjI/RIVER BASIN' from df
# df = df[df.iloc[:,1] != 'LEMBANGANSUNGAI/RIVER BASIN'].reset_index(drop=True)
# replace with null value column 0 with value 'NEGERI/STATE'
df.iloc[:,0] = df.iloc[:,0].replace('NEGERI/STATE', None)
# forward fill column 0
df.iloc[:,0] = df.iloc[:,0].ffill()
# only in case column 9 is null, move values in column 0 to column 8 to column 1 to column 9 but maintain if column 9 is not null
mask = df.iloc[:,9].isnull()
df[mask] = df[mask].shift(axis=1)
mask = df.iloc[:,9].isnull()
df[mask] = df[mask].shift(axis=1)
df.iloc[:,[0,1]] = df.iloc[:,[0,1]].ffill()

In [4]:
df.describe()

,state,river-basin,river,no-of-stations,2019-IKA/WQI,2019-category,2019-class,2020-IKA/WQI,2020-category,2020-class
count,34,34,34,34,34,34,34,34,34,34
unique,9,18,32,5,24,2,3,16,1,3
top,Johor,Sg. Tebrau,Sg. Rambai,1,54,T/P,III,59,T/P,III
freq,18,7,2,23,3,30,17,5,34,24


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   state           34 non-null     object
 1   river-basin     34 non-null     object
 2   river           34 non-null     object
 3   no-of-stations  34 non-null     object
 4   2019-IKA/WQI    34 non-null     object
 5   2019-category   34 non-null     object
 6   2019-class      34 non-null     object
 7   2020-IKA/WQI    34 non-null     object
 8   2020-category   34 non-null     object
 9   2020-class      34 non-null     object
dtypes: object(10)
memory usage: 2.8+ KB


In [6]:
df.to_excel('Jadual-2-3-sungai-tercemar-2020.xlsx', index=False)